In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/en-core-web-md/en_core_web_md/meta.json
/kaggle/input/en-core-web-md/en_core_web_md/__init__.py
/kaggle/input/en-core-web-md/en_core_web_md/en_core_web_md-3.4.1/tokenizer
/kaggle/input/en-core-web-md/en_core_web_md/en_core_web_md-3.4.1/LICENSES_SOURCES
/kaggle/input/en-core-web-md/en_core_web_md/en_core_web_md-3.4.1/LICENSE
/kaggle/input/en-core-web-md/en_core_web_md/en_core_web_md-3.4.1/README.md
/kaggle/input/en-core-web-md/en_core_web_md/en_core_web_md-3.4.1/meta.json
/kaggle/input/en-core-web-md/en_core_web_md/en_core_web_md-3.4.1/accuracy.json
/kaggle/input/en-core-web-md/en_core_web_md/en_core_web_md-3.4.1/config.cfg
/kaggle/input/en-core-web-md/en_core_web_md/en_core_web_md-3.4.1/attribute_ruler/patterns
/kaggle/input/en-core-web-md/en_core_web_md/en_core_web_md-3.4.1/vocab/vectors
/kaggle/input/en-core-web-md/en_core_web_md/en_core_web_md-3.4.1/vocab/key2row
/kaggle/input/en-core-web-md/en_core_web_md/en_core_web_md-3.4.1/vocab/vectors.cfg
/kaggle/input/en-core-we

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import make_scorer
from sklearn.model_selection import train_test_split, GridSearchCV
import spacy

nlp = spacy.load("../input/en-core-web-md/en_core_web_md/en_core_web_md-3.4.1")

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [3]:
summary_df = pd.read_csv('/kaggle/input/commonlit-evaluate-student-summaries/summaries_train.csv')
prompt_df = pd.read_csv('/kaggle/input/commonlit-evaluate-student-summaries/prompts_train.csv')

In [4]:
from string import punctuation as punc
from collections import defaultdict
import spacy
from string import punctuation as punc
from tqdm import tqdm
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances


def process_text(text):
    doc = nlp(text)
    lemmas = []
    stops = 0
    pos = defaultdict(int,{'VERB': 0, "NOUN": 0})
    char_length = len(text)
    word_vec = []

    for token in doc:
        if token.is_stop:
            stops += 1
        else:
            if token.pos_ == "VERB":
                pos[token.pos_] += 1
            elif token.pos_ == "NOUN":
                pos[token.pos_] += 1
            lemmas.append(token.lemma_.lower())
            word_vec.append(token.vector)

    word_vec = np.mean(word_vec, axis=0)
    pos = dict(pos)
    return ' '.join(lemmas), stops, pos['VERB'], pos['NOUN'], char_length, word_vec

def count_punc(text):
    return len([p for p in text if p in punc])

# Overlap Score
def overlap(sample):
    answer = set(sample['text'].lower().split())
    context = set(sample['prompt_text'].lower().split())
    intersection = len(context.intersection(answer))
    difference = len(context.difference(answer))
    union = len(context.union(answer))
    overlap_score = intersection/float(len(context))
    jaccard_similarity = intersection/union
    return intersection, difference, union, overlap_score, jaccard_similarity

def word2vec_similarity(sample, model):
    vector1 = np.mean([model.wv[word] for word in sample['context'].split() if word in model.wv.index_to_key], axis=0)
    vector2 = np.mean([model.wv[word] for word in sample['lemmas'].split() if word in model.wv.index_to_key], axis=0)
    dot_product = vector1.dot(vector2)
    similarity = cosine_similarity([vector1], [vector2])[0][0]
    distance = euclidean_distances([vector1], [vector2])[0][0]
    coef = np.corrcoef([vector1], [vector2])[0][1]
    return similarity, distance, dot_product, coef

In [5]:
#@title Best Score yet using spacy text vector on the summary text and context.
###@title Features: [context_word_vec, word_vec, similarity, word2vec, cols]

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import TruncatedSVD,PCA
from scipy.sparse import csr_matrix

def preprocess_data(summary_df, prompt_df, ngram_range=(1,1)):
    train_df = summary_df.merge(right=prompt_df, how='inner', on='prompt_id')

    print('processing text data')

    # Refactored text processing
    def process_texts(column_name):
        return zip(*train_df[column_name].apply(process_text).tolist())

    word_vec = np.zeros((train_df.shape[0], 300))
    context_word_vec = np.zeros((train_df.shape[0], 300))
    train_df['intersection'], train_df['difference'], train_df['union'], train_df['overlap_score'], train_df['jaccard_similarity'] = zip(*train_df.apply(overlap, axis=1).tolist())

    train_df['lemmas'],train_df['summary_stopwords'],train_df['summary_verbs'], train_df['summary_nouns'], train_df['length_of_summary'], word_vec[:] = process_texts('text')
    train_df['context'], train_df['context_stopwords'],train_df['context_verbs'], train_df['context_nouns'], train_df['length_of_context'], context_word_vec[:] = process_texts('prompt_text')
    train_df['length_of_summary'] = train_df['text'].str.count(' ')

    print('loading word2vec model......')
    sentences = train_df.apply(lambda x: x['context'].split() + x['lemmas'].split(), axis=1)
    model = Word2Vec(sentences, vector_size=512, window=5, min_count=1, workers=4)

    print('processing features.....')
    # Process other columns
    train_df['summary_punctuation'] = train_df['text'].apply(count_punc)

    cols = ['length_of_summary', 'summary_punctuation','summary_stopwords','summary_verbs','summary_nouns',
        'intersection','difference','union', 'overlap_score', 'jaccard_similarity',
        'context_stopwords','context_verbs','context_nouns','length_of_context','content', 'wording']

    print('applying vectorizer.....')
    # Vectorization
    global vectorizer
    vectorizer = TfidfVectorizer(sublinear_tf=True,lowercase=True,dtype=np.float32,stop_words='english',ngram_range=ngram_range)
    summary_transformed = vectorizer.fit_transform(train_df['lemmas'])

    print('scaling features.....')

    print('decomposing features.......')
    word2vec = np.asarray(train_df.apply(word2vec_similarity, axis=1, args=(model,)).to_list(),dtype=np.float32)
    similarity = np.zeros((train_df.shape[0], 3))
    for i, data in tqdm(train_df.iterrows(), 'processing similarity'):
        full_bags = vectorizer.transform([data['context']])
        summary_bags = vectorizer.transform([data['lemmas']])
        similarity[i, 0] = cosine_similarity(full_bags, summary_bags)[0][0]
        similarity[i, 1] = euclidean_distances(full_bags, summary_bags)[0][0]
        similarity[i, 2] = np.corrcoef(full_bags.toarray(), summary_bags.toarray())[0, 1]

    stacked_bag = np.hstack((context_word_vec, word_vec, similarity, word2vec, train_df[cols].values))

    print('collecting outputs.........')
    student_ids = train_df['student_id'].values

    return stacked_bag, student_ids

In [6]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import time

start_time = time.time()

stacked_bag, student_ids = preprocess_data(summary_df,prompt_df,ngram_range=(1,1))

end_time = time.time()
elapsed_time = end_time - start_time

print(f"Preprocessed function executed in: {elapsed_time:.2f} seconds")

processing text data
loading word2vec model......
processing features.....
applying vectorizer.....
scaling features.....
decomposing features.......


processing similarity: 7165it [00:28, 247.55it/s]

collecting outputs.........
Preprocessed function executed in: 1037.22 seconds


In [7]:
X_train,X_test,y_train,y_test = train_test_split(stacked_bag[:,:-2] ,stacked_bag[:,-2:],test_size=0.1,shuffle=True,random_state=11)

In [8]:
def mcrmse(y_true, y_pred):
    return np.mean(np.sqrt(np.mean((y_true - y_pred)**2, axis=0)))

mcrmse_scorer = make_scorer(mcrmse, greater_is_better=False)
best_params = {'max_depth': 11,'max_features': 0.6,'min_samples_split': 10,'n_estimators': 400}
model_rf = MultiOutputRegressor(RandomForestRegressor(**best_params,n_jobs=-1, random_state=42),n_jobs=-1)

In [9]:
model_rf.fit(X_train,y_train)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


MultiOutputRegressor(estimator=RandomForestRegressor(max_depth=11,
                                                     max_features=0.6,
                                                     min_samples_split=10,
                                                     n_estimators=400,
                                                     n_jobs=-1,
                                                     random_state=42),
                     n_jobs=-1)

In [10]:
y_preds = model_rf.predict(X_test)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [11]:
mcrmse(y_test,y_preds)

0.5101984542171801

In [12]:
from joblib import dump

dump(vectorizer, '/kaggle/working/vectorizer.pkl')
dump(model_rf, '/kaggle/working/model_rf.pkl')

['/kaggle/working/model_rf.pkl']

In [13]:
from joblib import load

model_rf = load('/kaggle/working/model_rf.pkl')
vectorizer = load('/kaggle/working/vectorizer.pkl')

In [14]:
def preprocess_data(summary_df, prompt_df, ngram_range=(1,1), vectorizer=vectorizer):
    train_df = summary_df.merge(right=prompt_df, how='inner', on='prompt_id')

    print('processing text data')

    # Refactored text processing
    def process_texts(column_name):
        return zip(*train_df[column_name].apply(process_text).tolist())

    word_vec = np.zeros((train_df.shape[0], 300))
    context_word_vec = np.zeros((train_df.shape[0], 300))
    train_df['intersection'], train_df['difference'], train_df['union'], train_df['overlap_score'], train_df['jaccard_similarity'] = zip(*train_df.apply(overlap, axis=1).tolist())

    train_df['lemmas'],train_df['summary_stopwords'],train_df['summary_verbs'], train_df['summary_nouns'], train_df['length_of_summary'], word_vec[:] = process_texts('text')
    train_df['context'], train_df['context_stopwords'],train_df['context_verbs'], train_df['context_nouns'], train_df['length_of_context'], context_word_vec[:] = process_texts('prompt_text')
    train_df['length_of_summary'] = train_df['text'].str.count(' ')

    print('loading word2vec model......')
    sentences = train_df.apply(lambda x: x['context'].split() + x['lemmas'].split(), axis=1)
    model = Word2Vec(sentences, vector_size=512, window=5, min_count=1, workers=4)

    print('processing features.....')
    # Process other columns
    train_df['summary_punctuation'] = train_df['text'].apply(count_punc)

    cols = ['length_of_summary', 'summary_punctuation','summary_stopwords','summary_verbs','summary_nouns',
        'intersection','difference','union', 'overlap_score', 'jaccard_similarity',
        'context_stopwords','context_verbs','context_nouns','length_of_context']

    print('applying vectorizer.....')
    # Vectorization
    summary_transformed = vectorizer.transform(train_df['lemmas'])

    print('scaling features.....')

    print('decomposing features.......')
    word2vec = np.asarray(train_df.apply(word2vec_similarity, axis=1, args=(model,)).to_list(),dtype=np.float32)
    similarity = np.zeros((train_df.shape[0], 3))
    for i, data in tqdm(train_df.iterrows(), 'processing similarity'):
        full_bags = vectorizer.transform([data['context']])
        summary_bags = vectorizer.transform([data['lemmas']])
        similarity[i, 0] = cosine_similarity(full_bags, summary_bags)[0][0]
        similarity[i, 1] = euclidean_distances(full_bags, summary_bags)[0][0]
        similarity[i, 2] = np.corrcoef(full_bags.toarray(), summary_bags.toarray())[0, 1]

    stacked_bag = np.hstack((context_word_vec, word_vec, similarity, word2vec, train_df[cols].values))

    print('collecting outputs.........')
    student_ids = train_df['student_id'].values

    return stacked_bag, student_ids

In [15]:
test_summary = pd.read_csv('/kaggle/input/commonlit-evaluate-student-summaries/summaries_test.csv')
test_prompts = pd.read_csv('/kaggle/input/commonlit-evaluate-student-summaries/prompts_test.csv')

In [16]:
test_df,ids = preprocess_data(test_summary, test_prompts)

processing text data
loading word2vec model......
processing features.....
applying vectorizer.....
scaling features.....
decomposing features.......


processing similarity: 4it [00:00, 233.08it/s]

collecting outputs.........


In [17]:
y_preds = model_rf.predict(test_df)

In [18]:
df_test = pd.DataFrame(y_preds,columns=['content','wording'],index=ids).reset_index()
df_test = df_test.rename({'index':'student_id'},axis=1)

In [19]:
df_test

,student_id,content,wording
0,000000ffffff,-1.163273,-0.796001
1,222222cccccc,-1.207142,-0.789528
2,111111eeeeee,-1.192468,-0.796646
3,333333dddddd,-1.179875,-0.806766


In [20]:
df_test.to_csv("submission.csv", index=False)